# CH04. Template
---
게시판을 사용하는 앱에서 핵심기능이 될 목록조회와 상세조회를 구현하는 법을 배우자.<br>
### 1. 목록조회
다음 페이지 요청시 등록한 Question들을 조회할 수 있도록 구현해 보자.<br>
``` http://localhost:8000/myapp ```<br>
 지금은 위 페이지를 요청하면 index 함수에서 HttpResponse에 두었던 문구가 출력된다. Question 목록이 출력되도록 myapp/views.py 파일의 index 함수를 다음과 같이 변경해 보자.<br><br>
``` myapp/views.py ```
```python
# ---------------------[ edit ]-----------------------
from django.shortcuts import render
from .models import Question
# from django.http import HttpResponse # 이건 일단 지웠다.

def index(request):
    """
    Questino 목록 출력
    """
    question_list = Question.objects.order_by('-create_date')
    context = {'question_list': question_list}
    return render(request, 'myapp/question_list.html', context)

# ----------------------------------------------------
```

HttpResponse로 단순히 문자열만 리턴했던 함수가 위처럼 변경되었다.<br><br>
일단 질문목록 데이터는 ``` Question.objects.order_by('-create_date') ```로 얻을 수 있다. order_by는 조회결과를 정렬하는 함수이다. ``` order_by('-create_date') ```의 의미는 작성일시 순인데 ``` - ```기호가 붙어 있으므로 작성일시의 역순으로 정렬하라는 의미이다.<br><br>


render 함수는 조회된 질문목록 데이터를 myapp/question_list.html 파일에 적용하여 HTML로 변환해 주는 함수이다. 여기서 사용된 myapp/question_list.html 파일을 템플릿이라고 부른다. 템플릿 파일은 HTML파일과 비슷하지만 장고에서 사용하는 태그들을 사용할 수 있는 HTML파일이다. <br><br>

context 딕셔너리는 페이지로 데이터를 넘겨주기 위해서 존재한다.<br><br>

<strong>1.1. 템플릿 디렉토리</strong><br>
render 함수에서 사용된 myapp/question_list.html 템플릿 파일을 작성해야 한다. 하지만 템플릿 파일을 작성하기 전에 템플릿 파일을 저장할 디렉토리를 먼저 만들어야 한다. 템플릿을 저장할 디렉토리는 mysite/setting.py 파일의 TEMPLATES 항목에 추가해 주어야 한다.<br>
``` mysite/settings.py ```
```python
TEMPLATES = [
    {
        ...
        'DIRS': [os.path.join(BASE_DIR, 'templates')],
    },
]
```
``` DIRS ```는 템플릿 디렉토리를 여러개 등록할 수 있도록 리스트로 되어 있다. myapp은 ``` os.path.join(BASE_DIR, 'templates') ```디렉토리만 등록하도록 하자.<br>
이제 템플릿 파일을 저장할 디렉토리의 경로는 다음과 같다.<br>
``` mysite/templates ``` (없으면 생성해라)<br>
>템플릿 파일의 관리는 템플릿 폴더내에서 myapp 폴더를 추가로 생성하여 사용하자.

>참고로 mysite라는 장고 프로젝트 생성시 mysite안에 mysite 디렉토리와 앱들 디렉토리, manage,py 파일 등이 있는데, BASE_DIR의 초기값은 제일 상위 mysite 디렉토리 경로이다.


<strong>1.2. 템플릿 파일</strong><br>
``` templates/myapp/question_list.html ```
```html
<!-- --------------------[ edit ]----------------------- -->
{% if question_list %}
    <ul>
    {% for question in question_list %}
        <li><a href="/myapp/{{ question.id }}/">{{ question.subject }}</a></li>
    {% endfor %}
    </ul>
{% else %}
    <p>질문이 없습니다.</p>
{% endif %}
<!-- --------------------------------------------------- -->
```

템플릿을 보면 ``` {% if question_list %} ```처럼 ```{% ``` 와 ``` %} ``` 로 둘러싸인 문장들을 볼 수 있는데 이러한 것들을 템플릿 태그라고 한다.<br><br>
question_list.html에 사용된 템플릿 태그들은 파이썬의 명령들과 대거 비슷하다.<br><br>


<strong>1.3. 템플릿 태그</strong><br>
장고에서 사용되는 템플릿 태그는 다음 3가지 유형만 알면 된다.
1. 분기

분기문 태그의 사용법은 다음과 같다.
```html
{% if 조건문1 %}
    <p>조건문1에 해당되는 경우</p>
{% elif 조건문2 %}
    <p>조건문2에 해당되는 경우</p>
{% else %}
    <p>조건문1, 2에 모두 해당되지 않는 경우</p>
{% endif %}
```

2. 반복

반복문 태그의 사용법은 다음과 같다.
```html
{% for item in list %}
    <p>순서: {{ forloop.counter }} </p>
    <p>{{ item }}</p>
{% endfor %}
```
템플릿 for문 안에서는 다음과 같은 ``` forloop ```객체를 사용할 수 있다.
>forloop.counter : 루프내의 순서로 1부터 표시<br>
>forloop.counter0 : 루프내의 순서로 0부터 표시<br>
>forloop.first : 루프의 첫번째 순서인 경우 True<br>
>forloop.last : 루프의 마지막 순서인 경우 True<br>

3. 객체 출력

객체를 출력하기 위한 태그의 사용법은 다음과 같다.
```html
{{ 객체 }}
```
객체에 속성이 있는 경우는 파이썬과 동일한 방법으로 도트(``` . ```)문자를 이용하여 표시하면 된다.
```html
{{ 객체.속성 }}
```
<br>
추가적으로 필요한 템플릿 문법이 있으면 장고 공식홈페이지를 참고하자.

[템플릿 관련 문서](https://docs.djangoproject.com/en/3.0/topics/templates/)


### 2. 상세조회
이제 목록조회 화면에 조회된 링크를 클릭했을때의 url매핑을 하고 상세조회 창을 띄어보자.<br><br>
<strong>2.1. URL 매핑</strong><br>
``` http://localhost:8000/myapp/2/ ```에 접속하는 것은 Question 모델 데이터 중 2라는 id값을 가지고 있는 데이터를 조회하는 것이다. 우선 URL을 매핑하자. myapp/urls.py를 수정한다.<br>
``` myapp/urls.py ```<br>
```python
from django.urls import path

from . import views

urlpatterns = [
    path('', views.index),
    path('<int:question_id>/', views.detail),
]
```
위의 코드를 보면 ``` path('<int:question_id>/', views.detail) ```라는 URL 매핑을 추가하였다. <br><br>

http://localhost:8000/myapp/2/ 와 같은 페이지가 요청되면 위 매핑룰에 의해<br>
http://localhost:8000/myapp/<int:question_id>/ 가 적용되어 question_id 에 2라는 값이 저장되고 ``` views.detail ``` 함수가 실행될 것이다. (여기서 int는 숫자값이 매핑됨을 의미한다.)
<br><br>
<strong>2.2. views.detail 함수 </strong><br>
URL에 매핑이 된 detail이라는 함수를 추가해야한다.<br>
``` myapp/views.py ```
```python
...
def detail(request, question_id):
    """
    myapp 내용 출력
    """
    question = Question.objects.get(id=question_id)
    context = {'questiopn': question}
    return render(request, 'myapp/question_detail.html', context)
...
```
index 함수와 크게 다른 부분은 없는데, detail 함수 호출시 전달되는 매개변수가 question_id가 하나 더 추가된 것에 주목한다. 매개변수 question_id에는 URL 매핑시 저장된 question_id가 전달된다.<br><br>
즉, http://localhost:8000/myapp/2/ 페이지가 요청되면 최종적으로 detail 함수의 매개변수 question_id에는 2라는 값이 전달되게 된다.<br><br>
그리고 여기서 question 이라는 변수에 받은 id값을 이용해서 question 변수를 받아서 render 함수로 넘겨준다.

<br><br>
<strong>2.3. question_detail.html </strong><br>
상세조회 화면에 해당되는 myapp/question_detail.html 템플릿을 다음처럼 작성하자.<br>
``` templates/myapp/question_detail.html ```
```html
<!-- -----------------[ edit ]------------------- -->
<h1>{{ question.subject }}</h1>

<div>
    {{ question.content }}
</div>
<!-- -------------------------------------------- -->
```